# Video to TIF

Video formats are usually difficult to read in Python so it is [recommended to convert them to '.tif' files](https://scikit-image.org/docs/stable/user_guide/video.html). This notebook shows how to use the library [OpenCV](https://docs.opencv.org/2.4/index.html) (Open Source Computer Vision) includes [VideoCapture](https://docs.opencv.org/2.4/modules/highgui/doc/reading_and_writing_images_and_video.html?highlight=videocapture) to read the video files and save them to tiff using the [tifffile](https://github.com/cgohlke/tifffile) library. When saving to tif, you can also compress the file to further reduce the file size. 

# Example data

Infrared video recordings of the mouse face.

Go to `Example_files` > `video_to_tif` > `FCM01a.avi`, `FCM02a.mp4`, download the files, and paste them into the 'Data_example' folder of the notebook after you create the paths. Alternatively, if you clone the repository, paste the `video_to_tif` folder into the new `Data_examples` directory. After that, you can run all the cells of the notebook. 


# Import the libraries


In [ ]:
import numpy as np

import re
import os
import sys


import cv2
import tifffile

# Create the paths

In [ ]:
# Comment out the below code for customizing the paths

notebook_name = 'video_to_tif'

# Data path to where you want to locate your folder
data_path = os.getcwd()  # Default: current directory where the Jupyter notebook is located

# Change the folder names accordingly
paths = {'data': data_path,
         'raw_data':  f'{data_path}/Data_examples/{notebook_name}/',
         'processed_data': f'{data_path}/Processed_data_examples/{notebook_name}/',
         'analysis': f'{data_path}/Analysis_examples/{notebook_name}/',         
         'plots': f'{data_path}/Analysis_examples/{notebook_name}/Plots/'}

# Make folders if they do not exist yet
for path in paths.values():
    os.makedirs(path, exist_ok=True)

# Define the function

Here is a template function that you can modify and expand to suit your needs, such as removing the downsampling and saving statements.

The function extracts individual frames for processing, iteratively downsampling (or not) based on the video's dimensions, converting them to grayscale, and adding them to a list. The frames are saved as TIFF images using the tifffile library. See the [tifffile GitHub](https://github.com/cgohlke/tifffile) for adding or modifying the saving options in the function.

In [ ]:
def video_to_tif(file_path, paths, experiment, downsample_factor=1, compression=None):
    """
    Converts videos to TIFF stacks with optional downsampling and compression options.
    Uses OpenCV (cv2) for loading and processing video frames, and Tifffile for saving TIFF stacks with compression.
    Args:
        file_path (str): The path to the video file.
        paths (dict): A dictionary containing paths for data and analysis.
        experiment: experiment name.
        downsample_factor (int, optional): The factor by which to downsample the video. Default is 1 (no downsampling).
        compression (str, optional): The compression method to use for saving the TIFF stack. Default: no compression.
    Returns:
        str: The path to the saved TIFF stack.
    """
    # Load the video file
    video = cv2.VideoCapture(file_path)

    # Get the video dimensions
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Calculate the new video dimensions
    new_width = int(width // downsample_factor)
    new_height = int(height // downsample_factor)

    # Create a list to store frames
    frames = []

    # Loop over the video frames and add them to the list
    while True:
        # Read the next frame from the video
        ret, frame = video.read()
        if not ret:
            break

        # Downsample the frame
        downsampled_frame = cv2.resize(frame, (new_width, new_height))

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(downsampled_frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray_frame)

    # Close the video capture object
    video.release()

    # Create folder in 'processed_data' if it does not exist
    filename = os.path.splitext(os.path.basename(file_path))[0]
    save_path = f"{paths['processed_data']}{experiment}"
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Save the TIFF stack using tifffile to preserve metadata
    save_path_file = f"{save_path}/{filename}_{new_height}px.tif"
    tifffile.imwrite(save_path_file, 
                     frames,
                     compression=compression,
                     dtype=np.uint8)
    
    return print(save_path_file)

# Convert the videos to tif

In the example below, I use `os.walk()` to loop through the file names in a directory tree by walking the tree either top-down or bottom-up. For each directory in the tree rooted at the directory `top` (including `top` itself), it yields a 3-tuple: `dirpath`, `dirnames`, and `filenames`. See the [Documentation](https://docs.python.org/3/library/os.html) for more details. You can also loop through specific "recordings" saved in a designated location (`recordings_path`). If a file is not found, the file path of the missing file will be printed. Additionally, you can use **if** statements or [regular expressions](https://docs.python.org/3/library/re.html) to search for specific filenames or file types.

The example data consists of 1-minute videos in '.avi' and '.mp4' formats.

**Note**: With `compression='jpeg'`, files take significantly longer to open in ImageJ. The 'zlib' compression seems to work well.



In [ ]:
experiments = ['FCM01', 'FCM02']

# Downsampling factor and compression for video files (Optional)
downsample_factor = 2  
compression = 'zlib'  

# Walk through the directory
for root, subdirs, files in os.walk(paths['raw_data']):
    for file in files:
        for experiment in experiments:
            if file.startswith(experiment): 
                file_path = f"{root}/{file}"
            
                video_to_tif(file_path, paths, experiment, downsample_factor, compression)
